## Imports

In [5]:
#Commonted out the below lines as the packages are already installed in the environment

# !pip install timm
# !pip install nibabel
# !pip install nilearn
# !pip install light-the-torch && ltt install torch
# !pip install torchio

import tensorflow as tf
#for neuroimaging data
import nibabel as nib
import nilearn as nilearn
import timm as timm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch 
import torchio as tio
import os



In [6]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is not available.


In [7]:
# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {torch.cuda.get_device_name()} for inference' if torch.cuda.is_available() else 'Using CPU for inference')

Using CPU for inference


## Importing the Dataset

In [8]:


root_dir = 'brown'

# List all subdirectories in the root directory
subdirectories = [x[0] for x in os.walk(root_dir)]

dataset = []

# Iterate over the subdirectories
for subdir in subdirectories:
    # Check if the subdirectory contains 'anat' in its path
    if 'anat' in subdir:
        # List all files in the subdirectory
        files = os.listdir(subdir)
        # Iterate over the files
        for file in files:
            # Check if the file is a .nii.gz file
            if file.endswith('.nii.gz'):
                file_path = os.path.join(subdir, file)
                # Load the Nifti image using TorchIO
                subject = tio.Subject(
                    mri=tio.ScalarImage(file_path)
                )
                dataset.append(subject)

# Print the number of images in the dataset
print('Number of Images in Dataset: ' + str(len(dataset)))

# # Load NIfTI image using TorchIO
# image_path = 'path/to/your/nifti/image.nii.gz'
# subject = tio.Subject(
#     mri=tio.ScalarImage(image_path)
# )

# # Transform the image (e.g., resample, normalize)
# transform = tio.Compose([
#     tio.Resample(4),  
#     tio.ZNormalization(),
# ])
# transformed_subject = transform(subject)

# # Convert to PyTorch tensor
# tensor = transformed_subject.mri.data 


Number of Images in Dataset: 26


In [9]:
# import os
# import nibabel as nib

# root_dir = 'brown'

# # List all subdirectories in the root directory
# subdirectories = [x[0] for x in os.walk(root_dir)]

# dataset = []

# # Iterate over the subdirectories
# for subdir in subdirectories:
#     # Check if the subdirectory contains 'anat' in its path
#     if 'anat' in subdir:
#         # List all files in the subdirectory
#         files = os.listdir(subdir)
#         # Iterate over the files
#         for file in files:
#             # Check if the file is a .nii.gz file
#             if file.endswith('.nii.gz'):
#                 file_path = os.path.join(subdir, file)
#                 # Load the Nifti image
#                 data = nib.load(file_path)
#                 # Append the image data to the dataset
#                 dataset.append(data)
         
# # Print the number of images in the dataset
# print('Number of Images in Dataset: ' + str(len(dataset)))

## Data Profiling

In [10]:
dataset[0].mri.data


AttributeError: 'Subject' object has no attribute 'data'

In [ ]:
# # Access data array and metadata
# data = dataset[22].get_fdata()
# affine = dataset[22].affine 
# header = dataset[22].header

# # Display basic information
# print(f'Image dimensions: {data.shape}')
# print(f'Voxel dimensions: {header.get_zooms()}')

AttributeError: 'Subject' object has no attribute 'data'

In [11]:
# Display the image

plt.imshow(data[:, :, 90], cmap='gray')
plt.axis('on')



NameError: name 'data' is not defined

## Preprocessing

In [ ]:
# Resample the dataset

from nilearn.image import resample_img

# Define the target voxel dimensions
target_voxel_dimensions = (1.0, 1.0, 1.0)

# Resample the dataset to the target voxel dimensions
resampled_dataset = []
for data in dataset:
    resampled_data = resample_img(data, target_affine=np.diag(target_voxel_dimensions))
    resampled_dataset.append(resampled_data)

# Display the resampled image
resampled_data = resampled_dataset[22].get_fdata()
plt.imshow(resampled_data[:, :, 90], cmap='gray')
plt.axis('on')

# Load the labels
labels = pd.read_csv('brown/participants.tsv')
labels.head()


/var/folders/hr/f5tx3_6x0qj4kc28vj9rp1hm0000gn/T/ipykernel_2156/4211580262.py:11: UserWarning: Casting data from int16 to float32
  resampled_data = resample_img(data, target_affine=np.diag(target_voxel_dimensions))
/var/folders/hr/f5tx3_6x0qj4kc28vj9rp1hm0000gn/T/ipykernel_2156/4211580262.py:11: UserWarning: Casting data from int16 to float32
  resampled_data = resample_img(data, target_affine=np.diag(target_voxel_dimensions))
/var/folders/hr/f5tx3_6x0qj4kc28vj9rp1hm0000gn/T/ipykernel_2156/4211580262.py:11: UserWarning: Casting data from int16 to float32
  resampled_data = resample_img(data, target_affine=np.diag(target_voxel_dimensions))
/var/folders/hr/f5tx3_6x0qj4kc28vj9rp1hm0000gn/T/ipykernel_2156/4211580262.py:11: UserWarning: Casting data from int16 to float32
  resampled_data = resample_img(data, target_affine=np.diag(target_voxel_dimensions))
/var/folders/hr/f5tx3_6x0qj4kc28vj9rp1hm0000gn/T/ipykernel_2156/4211580262.py:11: UserWarning: Casting data from int16 to float32
  resa

KeyboardInterrupt: 

## Loading the Pre-Trained ResNet Model

In [ ]:
import torch
from torchvision.models.video import r3d_18

# Load the pre-trained model
model = r3d_18(pretrained=True)

# Modify the last layer
num_classes = 2  # replace with the number of classes in your dataset
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Print the model
model

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):